In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import mne
import entropy as ent
import neurokit2 as nk
from scipy.signal import welch
from nolds import lyap_r
import antropy as ant

In [10]:
participantsInfo = pd.read_table('./ds004504/participants.tsv')
#Getting all the groups
A_sub = participantsInfo[participantsInfo["Group"] == "A"]["participant_id"].tolist()
C_sub = participantsInfo[participantsInfo["Group"] == "C"]["participant_id"].tolist()
F_sub = participantsInfo[participantsInfo["Group"] == "F"]["participant_id"].tolist()

In [4]:
freq_bands = {
    "Delta": (0.5, 4),
    "Theta": (4, 8),
    "Alpha": (8, 12),
    "Beta": (12, 30),
}

import pandas as pd
import numpy as np 

def generate_subject_dataframe(num_subjects: int) -> pd.DataFrame:
    """
    Generates a DataFrame where each subject has 19 channels, 
    and each channel has 4 bands (Delta, Theta, Alpha, Beta).
    
    Parameters:
        num_subjects (int): Number of subjects to include in the DataFrame.
    
    Returns:
        pd.DataFrame: A DataFrame with hierarchical indexing for subjects, channels, and bands for each window.
    """
    subjects = [f"sub-{i+1:03d}" for i in range(num_subjects)]  # Format subject IDs as sub-XXX
    channels = [f"Channel-{i+1}" for i in range(19)]
    bands = ["Delta", "Theta", "Alpha", "Beta"]
    
    data = []
    for subject in subjects:
        for channel in channels:
            for band, val in freq_bands.items():
                data.append([subject, channel, band, []])
    
    df = pd.DataFrame(data, columns=["Subject", "Channel", "Band", "Window Avg"])

    #the .setIndex is creating a multiIndex DataFrame
    return df.set_index(["Subject", "Channel", "Band"])

# Example usage:
df = generate_subject_dataframe(5)  # Generates a DataFrame for 5 subjects
print(df)

                         Window Avg
Subject Channel    Band            
sub-001 Channel-1  Delta         []
                   Theta         []
                   Alpha         []
                   Beta          []
        Channel-2  Delta         []
...                             ...
sub-005 Channel-18 Beta          []
        Channel-19 Delta         []
                   Theta         []
                   Alpha         []
                   Beta          []

[380 rows x 1 columns]


In [5]:
def derivativesPath(sub): #dericatives is the preprocessed data
    return f"ds004504/{sub}/eeg/{sub}_task-eyesclosed_eeg.set"



NUM_SUBJECTS = 3
# Use the function inside the groups dictionary
groups = {
    'Alzeimers': generate_subject_dataframe(NUM_SUBJECTS),
    'Control': generate_subject_dataframe(NUM_SUBJECTS),
    'Dementia': generate_subject_dataframe(NUM_SUBJECTS),
}

# Print the DataFrame for 'Alzeimers'

WINDOW_LENGTH = 1.5
STEP_SIZE = .75 #50% sliding window


In [7]:
# ** this is the part where we would do a for loop per group, per subject in that group **

In [8]:
#*this data would be from forloop variables ... for group in groups ... for subject in NUM_SUBJECTS:
group = 'Alzeimers'
index = 0

In [37]:
current_sub = F_sub[index]
raw_sub = mne.io.read_raw_eeglab(derivativesPath(current_sub), preload=True)
sfreq = raw_sub.info['sfreq']

In [38]:
start_times = np.arange(0, raw_sub.times[-1] - WINDOW_LENGTH, STEP_SIZE)
# a_window_start, a_window_end = (window_index*STEP_SIZE, window_index*STEP_SIZE + WINDOW_LENGTH) 

In [39]:
# options are welch, multitaper and TFR (if we do per epoch) 
psd_sub = raw_sub.compute_psd(method='welch', fmin=0.5, fmax=30)
#psd_sub.plot()

Effective window size : 4.096 (s)


In [42]:
sub_delta = psd_sub.get_data(picks='Fp1', fmin=0.5, fmax = 4 ) 
print(sub_delta)

[[1.06630219e-09 6.43667644e-10 4.46297007e-10 3.21515806e-10
  2.45606684e-10 1.73485674e-10 1.46972160e-10 1.18401268e-10
  8.94080491e-11 6.28021275e-11 4.77541212e-11 5.05580573e-11
  3.89099882e-11 3.01063308e-11]]


In [43]:
print(sub_delta.mean(axis=-1)[0])

2.4869907872974233e-10


In [32]:
print(psd_sub.info['ch_names'])

['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
